In [2]:
from google import colab
colab.drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# ▶ CatBoost 설치 (최신 버전 추천)
!pip install -q catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.8 MB/s eta 0:00:00


In [5]:
import catboost
print(catboost.__version__)

# GPU 인식 확인
!nvidia-smi

1.2.8
/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
# ▶ 1. 필수 라이브러리
import pandas as pd
import numpy as np
import pickle
from catboost import CatBoostClassifier, Pool

# ▶ 2. 저장된 항목 로드
base_path = '/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/catboost'

X_vif = pd.read_csv(f'{base_path}/X_vif.csv')
y_vif_encoded = pd.read_csv(f'{base_path}/y_vif_encoded.csv')['Segment_encoded']

with open(f'{base_path}/cat_features.pkl', 'rb') as f:
    cat_features = pickle.load(f)

with open(f'{base_path}/label_encoder.pkl', 'rb') as f:
    le = pickle.load(f)

# ▶ 3. 최종 모델 학습
final_model = CatBoostClassifier(
    loss_function="MultiClass",
    eval_metric="MultiClass",
    task_type="GPU",
    learning_rate=0.01,
    iterations=70000,
    l2_leaf_reg=50,
    random_seed=42,
    od_type="Iter",
    depth=5,
    early_stopping_rounds=15000,
    border_count=64,
    verbose=100
)

final_pool = Pool(X_vif, y_vif_encoded, cat_features=cat_features)
final_model.fit(final_pool)

# ▶ (선택) 학습된 모델 저장
final_model.save_model(f'{base_path}/final_catboost_model.cbm')

0:	learn: 1.5800694	total: 59.5ms	remaining: 1h 9m 28s
100:	learn: 0.6165676	total: 3.95s	remaining: 45m 34s
200:	learn: 0.4203656	total: 8.59s	remaining: 49m 42s
300:	learn: 0.3532089	total: 13.4s	remaining: 51m 35s
400:	learn: 0.3254153	total: 17.1s	remaining: 49m 28s
500:	learn: 0.3116570	total: 23.7s	remaining: 54m 49s
600:	learn: 0.3032300	total: 28.5s	remaining: 54m 50s
700:	learn: 0.2972546	total: 32.1s	remaining: 52m 55s
800:	learn: 0.2926510	total: 35.7s	remaining: 51m 27s
900:	learn: 0.2889744	total: 41.3s	remaining: 52m 43s
1000:	learn: 0.2858108	total: 44.9s	remaining: 51m 36s
1100:	learn: 0.2831336	total: 48.5s	remaining: 50m 38s
1200:	learn: 0.2807606	total: 54s	remaining: 51m 32s
1300:	learn: 0.2785816	total: 57.7s	remaining: 50m 45s
1400:	learn: 0.2766883	total: 1m 1s	remaining: 50m 2s
1500:	learn: 0.2748536	total: 1m 6s	remaining: 50m 42s
1600:	learn: 0.2732244	total: 1m 10s	remaining: 50m 2s
1700:	learn: 0.2716232	total: 1m 14s	remaining: 49m 32s
1800:	learn: 0.270247

In [ ]:
# ▶ 테스트 데이터 불러오기
test = pd.read_csv('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/피처 선택/test_피처43.csv')

# ▶ ID 컬럼 저장 후 피처 정리
test_id = test['ID']
X_test = test[X_vif.columns]  # 컬럼 순서 맞추기
X_test = X_test.drop(columns=['ID'], errors='ignore')

# ▶ Pool 생성
test_pool = Pool(data=X_test, cat_features=cat_features)

# ▶ 예측 (class index → 문자 라벨로 복원)
preds_class = final_model.predict(test_pool)
preds_labels = le.inverse_transform(preds_class.astype(int).flatten())

# ▶ 제출 파일 생성
submission = pd.DataFrame({
    'ID': test_id,
    'Segment': preds_labels
})

# ▶ 저장
submission_path = '/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/catboost/submission_catboost.csv'
submission.to_csv(submission_path, index=False)

In [ ]:
sub = pd.read_csv(submission_path)
sub

NameError: name 'pd' is not defined

In [ ]:
# 마지막 100,000개만 추출
submission_tail = submission.tail(100000)

# 제출 파일 저장
submission_tail.to_csv(
    '/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/catboost/submission_tail.csv',
    index=False,
    encoding='utf-8-sig'
)

print("✅ 마지막 100,000개로 제출 파일 저장 완료!")

✅ 마지막 100,000개로 제출 파일 저장 완료!


In [7]:
sub_tail = pd.read_csv('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/catboost/submission_tail.csv')
sub_tail

NameError: name 'pd' is not defined

In [8]:
import pandas as pd

In [9]:
test1 = pd.read_csv('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/catboost/submission_catboost.csv')
test1

,ID,Segment
0,TEST_00000,E
1,TEST_00001,E
2,TEST_00002,D
3,TEST_00003,E
4,TEST_00004,E
...,...,...
599995,TEST_99995,E
599996,TEST_99996,E
599997,TEST_99997,E
599998,TEST_99998,C


In [10]:
# ID별 Segment 다수결
submit_df = (
    test1
    .groupby('ID')['Segment']
    .agg(lambda x: x.value_counts().idxmax())
    .reset_index()
)

# 저장
submit_df.to_csv('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/catboost/submission_winner.csv', index=False, encoding='utf-8-sig')

In [12]:
test_win = pd.read_csv('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/catboost/submission_winner.csv')
test_win

,ID,Segment
0,TEST_00000,E
1,TEST_00001,E
2,TEST_00002,E
3,TEST_00003,E
4,TEST_00004,E
...,...,...
99995,TEST_99995,E
99996,TEST_99996,E
99997,TEST_99997,E
99998,TEST_99998,C
